## **CA2-AI**
# **Amir-Hossein Shahgholi(810199441)**

### Part1-Genetic

In first part we want to implementing a genetic algorithm for portfolio management:

At first we should create our Portfolio class to representation our chromosomes.

In [334]:
class Portfolio:
    data = list
    fitness_result = int
    
    def __init__(self, data):
        self.data = data

In our Chromosome class we have a list of data that represent amount of each shares in our Protfolio.

Now we should create our population first. We fill data with some random numbers that sum of them will become 1.
To do this we first generate some random numbers and then normalize the numbers.</br>
Below function will do that for us.

In [335]:
import random

ZERO_PERCENT = 0.6
def create_sample_data(number_of_data):
    numbers = []
    for _ in range(number_of_data):
        if (random.random()< ZERO_PERCENT):
            numbers.append(0)
        else:
            random_no = random.random()
            numbers.append(random_no)
    total = sum(numbers)
    numbers = [x/total for x in numbers]
    return numbers

So we create sample portfolio with randoms data that represent our population.

In [336]:
NUMBER_OF_CHOROMOSOME_IN_POPULATION = 1000
NUMBER_OF_SHARES = 400

population = []

for _ in range(NUMBER_OF_CHOROMOSOME_IN_POPULATION):
    sample_data = create_sample_data(NUMBER_OF_SHARES)
    temp_portfolio = Portfolio(data=sample_data)
    population.append(temp_portfolio)

Now create our population with sample random data.

In the next step we should define a fitness function to calculate a fitness value for each portfolio.
For fitness function first we find total_returns of our portfolio and then we should find risk of our portfolio using the portfolio's standard deviation and then we calculate a fitness value with this formula: **fitness = (portfolio_return - risk_penalty * portfolio_risk) / return_penalty**

In [337]:
import numpy as np

risk_penalty = 0.6
return_penalty = 10


def calculate_fitness(returns: list, risks: list, portfolio):
    fitness = 0
    returns = np.array(returns, dtype=np.float64)
    risks = np.array(risks, dtype=np.float64)
    shares_percent = np.array(portfolio.data, dtype=np.float64)
    num_zeros = np.count_nonzero(shares_percent == 0.0)
    if (NUMBER_OF_SHARES - num_zeros) <30:
        fitness -= 0.1
    portfolio_return = np.dot(returns, shares_percent)
    portfolio_risk = np.sqrt(np.dot(shares_percent**2, risks))
    fitness += (portfolio_return - risk_penalty * portfolio_risk) / return_penalty
    return fitness

To find crossover of two chromosomes we use uniform crossover and then after finding child's data we normalize the data to have some of one in child's data. 

In [338]:
def crossover(portfolio1, portfolio2) -> Portfolio:
    pdata1 = portfolio1.data 
    pdata2 = portfolio2.data
    child_data = []
    for i in range(NUMBER_OF_SHARES):
        if random.randint(0, 1) == 0:
            child_data.append(pdata1[i])
        else:
            child_data.append(pdata2[i])
    total = sum(child_data)
    child_data = [x/total for x in child_data]
    return Portfolio(child_data)

In [339]:
def mutation(portfolio) -> Portfolio:
    portfolio.data = create_sample_data(len( portfolio.data))
    return portfolio

Read the sample data.

In [340]:
import csv

returns = []
risks = []

with open('sample.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        returns.append(row['return'])
        risks.append(row['risk'])

In [341]:
def print_portfolio_data(portfolio, returns, risks):
    returns = np.array(returns, dtype=np.float64)
    risks = np.array(risks, dtype=np.float64)
    shares_percent = np.array(portfolio.data, dtype=np.float64)
    portfolio_return = np.dot(returns, shares_percent)
    portfolio_risk = np.sqrt(np.dot(shares_percent**2, risks))
    print(f"Fitness: {portfolio.fitness_result}")
    print(f"Return: {portfolio_return}")
    print(f"Risk: {portfolio_risk}")
    print(f"Data: {portfolio.data}")

In [342]:
PC = 0.9
PM = 0.1
for cal in population:
    cal.fitness_result = calculate_fitness(returns, risks, cal)
population = sorted(population, key=lambda portfolio: portfolio.fitness_result)
while True:
    population_no = len(population)
    parents = [population[i] for i in random.choices(range(population_no), weights=range(population_no), k=2)]
    if random.random() <= PC:
        new_child = crossover(parents[0], parents[1])
        new_child.fitness_result = calculate_fitness(returns, risks, new_child)
        population.pop(0)
        for z, chrom in enumerate(population):
            if chrom.fitness_result > new_child.fitness_result:
                population.insert(z, new_child)
                break
        else:
            population.append(new_child)
    if random.random() <= PM:
        random_chrom_index = random.randint(0,len(population)-1)
        population[random_chrom_index] = mutation(population[random_chrom_index])
    if population[-1].fitness_result > 1:
        break
print_portfolio_data(population[-1], returns, risks)
    

Fitness: 1.0022745844807193
Return: 10.226664968023783
Risk: 0.33986520536098275
Data: [0.0, 0.0, 0.0005826942984575442, 0.002068653951320995, 0.0, 0.0, 0.0, 0.0, 0.1605172344240528, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.028444078481902925, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0029323286967227373, 0.0, 0.0018403305559641185, 0.0, 0.0006178741081572541, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0002787354677074338, 0.0, 0.0, 0.0, 0.0, 0.038561830950611324, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11620988307745131, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0024091882250720834, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.023356445705524